## Project Introduction


* **Project Name:** Intelligent Lag Rate Monitoring Project Suitable for Video Company



* **Project Background:** User viewing experience is the lifeline of a video company, and playing lag is the number one killer affecting user experience. Users may watch choppy videos when using our products. As a data analyst, we should monitor the data of lag rate and other core indicators, use the anomaly detection algorithm to automatically find abnormal points, intelligently analyze the causes based on the root cause analysis algorithm, and form a daily intelligent monitoring report email, so as to escort the lifeline of the company.




* **Goal:** Design an intelligent Lag Rate Monitoring system which can achieve automatically fetch data from MySQL everyday, and judge whether the Lag Rate of the day is abnormal. If the value is normal, then perform routine analysis, analysizing Lag Rate from different dimensions, like calculating Lag Rate of different CDN providers and visualizing them accordingly. If the value is abnormal, then use root cause analysis algorithm to further position causes and output abnormality analysis report. Finally, form a complete analysis daily report and send through email.


* **Column Description:** 
* user_plat: what system user is using, like ios or android
* cdn_name: CDN(Content Delivery Network) providers.
* CDN is a geographically distributed network of proxy servers and their data centers. The goal is to provide high availability and performance by distributing the service spatially relative to end users.
[![63GBu9.png](https://s3.ax1x.com/2021/03/09/63GBu9.png)](https://imgtu.com/i/63GBu9)
* roomid: id of video
* province: province of user
* isp: the operator of user
* lag_people: Total number of users with lag problem in the day
* all_people: Total number of users in the day

* Lag Rate: key indicator = lag_people / all_people 
* For example, if Lag Rate in 2021-01-03 is 5%, it means 5 out of every 100 users that day have watched choppy videos.

## Work Flow
[![7CNDKK.png](https://s4.ax1x.com/2022/01/08/7CNDKK.png)](https://imgtu.com/i/7CNDKK)

In [1]:
### Import library

import pandas as pd 
import numpy as np
import pymysql
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False 
import datetime
from datetime import  timedelta
from fbprophet import Prophet
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import random
import datetime
import pyecharts
# pyecharts.globals._WarningControl.ShowWarning = False
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot
import smtplib
import email
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.header import Header

# access data from mysql
def mysql_datagain(day):

    db = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='12345',db='report')
    cursor =db.cursor()
    start_time = (datetime.datetime.strptime(day,'%Y-%m-%d') - timedelta(days=22)).strftime('%Y-%m-%d')
    sql = '''select * from lag_rate_detail where day between %s and %s'''
    cursor.execute(sql,(start_time,day))
    data = cursor.fetchall()
    col = [i[0] for i in cursor.description]
    df = pd.DataFrame(list(data),columns=col)
    db.close()
    return df

# Calculate lag rate of every day
def dataprocess(df):

    df['lag_people'] = df['lag_people'].astype(np.float).astype(int)
    df['all_people'] = df['all_people'].astype(np.float).astype(int)
    
    df_kpi = df.groupby('day',as_index=False)[['lag_people','all_people']].sum()
    df_kpi['lag_rate']  = df_kpi['lag_people'] / df_kpi['all_people']
    return df_kpi

# identify abnormal points with video lag problem using Prophet
def prophet_error(df,day):
    '''
    goal：judge whether the lag rate of the day is abnormal
    
    df : data after running function dataprocess()
    '''
    
    #transfer data into format Prophet needs
    df = df[['day','lag_rate']]
    df.columns = ['ds','y']
    
    m_e = Prophet(interval_width = 0.95)
    m_e.fit(df)
    future_e = m_e.make_future_dataframe(periods=0)
    forecast_e = m_e.predict(future_e)
    
    df['yhat_lower'] = forecast_e['yhat_lower']
    df['yhat_upper'] = forecast_e['yhat_upper']
    day_error = list(df[((df['y'] - df['yhat_lower'])<0) | ((df['y'] - df['yhat_upper'])>0)]['ds'])
    
    result = 1 if day in day_error else 0 
    return result

# Drill Down to analyze video lag reason from different dimensions -- text part
def analyse_textmake(df,day):
    '''
    goal: prepare description text in email
    df: original data from mysql
    '''
    df = dataprocess(df)
    
    
    # calculate lag rate of the day and yesterday and 7 days ago to make comparison
    today_lagrate = round(df[df['day']==day]['lag_rate'].values[0],4)
    yesterday = (datetime.datetime.strptime(day,'%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')
    last7day = (datetime.datetime.strptime(day,'%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')
    yesterday_lagrate = round(df[df['day']==yesterday]['lag_rate'].values[0],4)
    last7day_lagrate = round(df[df['day']==last7day ]['lag_rate'].values[0],4)
    
    # Compared with yesterday{increased by/decreased by/was flat}xxx%
    yesterday_dif = []
    if today_lagrate <= yesterday_lagrate:
        yes_text = 'decreased by'
        yesterday_dif.append(round((yesterday_lagrate - today_lagrate)*100/yesterday_lagrate,2))

    elif today_lagrate >= yesterday_lagrate:
        yes_text = 'increased by'
        yesterday_dif.append(round((today_lagrate-yesterday_lagrate)*100/yesterday_lagrate,2))
    else:
        yes_text = 'was flat'
        yesterday_dif.append('')
 
    # Compared with 7 days ago{increased by/decreased by/was flat}xxx%
    last7day_dif = []
    if today_lagrate <= last7day_lagrate:
        last7d_text = 'decreased by'
        last7day_dif.append(round((last7day_lagrate - today_lagrate)*100/last7day_lagrate,2))
    elif today_lagrate >= yesterday_lagrate:
        last7d_text = 'increased by'
        last7day_dif.append(round((today_lagrate-last7day_lagrate)*100/last7day_lagrate,2))
    else:
        last7d_text = 'was flat'
        last7day_dif.append('')
    
    # Text
    text = "Today is {}. The core indicator Video Lag Rate is {}, {} {}% compared with yesterday, {} {}% compared with last week. See more details from following tables and charts.".format(day,str(today_lagrate*100)+"%",yes_text,yesterday_dif[0],last7d_text,last7day_dif[0])
    return text

# Drill Down to analyze video lag reason from different dimensions -- table part
def analyse_tablemake(df,day):
    '''
    goal：building cdn Quality Ranking Table
    cdn: companies that provide service for CDN(Content Delivery Network)
    df：original data from mysql
    '''
    df_today = df[df['day']==day]
    df_cdn = df_today.groupby('cdn_name',as_index=False)[['lag_people','all_people']].sum()
    df_cdn['lag_rate'] = df_cdn['lag_people'] / df_cdn['all_people']
    df_cdn['rank'] = df_cdn['lag_rate'].rank().astype(int)
    df_cdn['lag_rate'] = round(df_cdn['lag_rate']*100,2).astype(str) +'%'
    df_cdn.columns = ['cdn_name','Number of users with lag','Number of users','Lag Rate','Quality Rank']
    df_cdn = df_cdn.sort_values('Quality Rank')
    return df_cdn

# Drill Down to analyze video lag reason from different dimensions -- charts part
def analyse_picturemake(df,day):
    '''
    goal：making analysis charts in email
    df：original data from mysql
    '''

    
    rich_text = {
        "a": {"color": "#999", "lineHeight": 22, "align": "center"},
        "abg": {
            "backgroundColor": "#e3e3e3",
            "width": "100%",
            "align": "right",
            "height": 22,
            "borderRadius": [4, 4, 0, 0],
        },
        "hr": {
            "borderColor": "#aaa",
            "width": "100%",
            "borderWidth": 0.5,
            "height": 0,
        },
        "b": {"fontSize": 16, "lineHeight": 33},
        "per": {
            "color": "#eee",
            "backgroundColor": "#334455",
            "padding": [2, 4],
            "borderRadius": 2,
        },
    }
    
    df_kpi = dataprocess(df)
    
    # Trend Chart of Lag Rate in Recent 20 day
    x_data = list(df_kpi['day'])
    y_data = list(df_kpi['lag_rate'].round(3))
    line = Line(init_opts=opts.InitOpts(theme='light',bg_color=JsCode(bg_color_js),width='700px',height='350px'))
    line.add_xaxis(x_data)
    line.add_yaxis('Lag Rate',y_data)
    line.set_series_opts(label_opts=opts.LabelOpts(is_show=False),itemstyle_opts=opts.ItemStyleOpts(color='red',border_color='black'),
                        markarea_opts=opts.MarkAreaOpts(data=[opts.MarkAreaItem(name="Spring Festival", x=("2021-02-11","2021-02-18"))]))
    line.set_global_opts(legend_opts=opts.LegendOpts(is_show=False),title_opts=opts.TitleOpts(title="Trend Chart of Lag Rate in Recent 20 days"))
    make_snapshot(snapshot,line.render(),"1.png")
    
    # Number of Total Users and Users with Lag Problem in Recent 20 Days
    y_data1 = list(df_kpi['lag_people'])
    y_data2 = list(df_kpi['all_people'])
    bar = Bar(init_opts=opts.InitOpts(theme='light', bg_color=JsCode(bg_color_js),width='700px', height='350px'))
    bar.add_xaxis(x_data)
    bar.add_yaxis('Number of users', y_data2)
    bar.add_yaxis('Number of users with lag', y_data1)
    bar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    bar.set_global_opts(title_opts=opts.TitleOpts(title="Number of Total Users and Users with Lag \nin Recent 20 Days"),legend_opts=opts.LegendOpts(pos_right=10))
    make_snapshot(snapshot,bar.render(),"2.png")
    
    # User Distribution by Platform in the day
    df_today = df[df['day']==day]
    df_plat_pie = df_today.groupby('user_plat',as_index=False)['all_people'].sum()
    x_plat_data = list(df_plat_pie ['user_plat'])
    y_plat_data = list(df_plat_pie ['all_people'])
    pie_plat = (Pie(init_opts=opts.InitOpts(theme='infographic',bg_color=JsCode(bg_color_js),width='700px', height='350px'))
           .add('Number of Users', [list(z) for z in zip(x_plat_data, y_plat_data)],
           label_opts=opts.LabelOpts(
                         formatter="\n\n\n{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}\n{per|{d}%}    ",
                         rich=rich_text))
           )
    pie_plat.set_global_opts(title_opts=opts.TitleOpts(title="User Distribution by Platform in %s"%day),legend_opts=opts.LegendOpts(pos_right=10))
    make_snapshot(snapshot,pie_plat.render(),"3.png")
    
    # User Distribution by Operator in the day
    df_isp_pie = df_today.groupby('isp',as_index=False)['all_people'].sum()
    x_isp_data = df_isp_pie['isp']
    y_isp_data = df_isp_pie['all_people']
    pie_isp = (Pie(init_opts=opts.InitOpts(theme='essos',bg_color=JsCode(bg_color_js),width='700px', height='350px'))
           .add('Number of Users', [list(z) for z in zip(x_isp_data, y_isp_data)],
           label_opts=opts.LabelOpts(position='outsiede',
                         formatter="\n\n\n\n\n\n{a|{a}}{abg|}\n{hr|}\n       {b|{b}: }{c}  {per|{d}%}       ",
                         rich=rich_text))
           )
    pie_isp.set_global_opts(title_opts=opts.TitleOpts(title="User Distribution by Operator in %s"%day),legend_opts=opts.LegendOpts(pos_right=10))
    make_snapshot(snapshot,pie_isp.render(),"4.png")
    
    # User Distribution by Province in the day
    df_prov = df_today.groupby('province',as_index=False)['all_people'].sum()
    df_prov = df_prov.sort_values('all_people')
    x_data = list(df_prov['province'])
    y_data = list(df_prov['all_people'])
    bar2 = Bar(init_opts=opts.InitOpts(theme='roma', bg_color=JsCode(bg_color_js),width='700px', height='350px'))
    bar2.add_xaxis(x_data)
    bar2.add_yaxis('Number of Users', y_data)
    bar2.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    bar2.set_global_opts(title_opts=opts.TitleOpts(title="User Distribution by Province in %s"%day),legend_opts=opts.LegendOpts(is_show=False))
    bar2.reversal_axis()
    make_snapshot(snapshot,bar2.render(),"5.png")

    
# make analysis report content with html+css

#cdn Quality Ranking Table
def generate_html(raw_html):
    html = """
    <head>
    <style type="text/css">
    .tg {border-collapse:collapse;border-spacing:0;}
    .tg td{font-family:Arial, sans-serif;font-size:16px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;text-align:center;}
    .tg th{font-family:Arial, sans-serif;font-size:20px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
    .tg .tg-baqh{font-weight:bold;border-color:inherit;text-align:left;vertical-align:top;background:#8fbc8f;text-align:center;}

    </style>
    </head>
    <body>
    <div style="font-size:20px;font-weight:bold">Video Quality and Lag Rate of CDN providers</div>
    <table class="tg">
             <tr>
                <th class="tg-baqh">cdn</th>
                <th class="tg-baqh">Number of users with lag</th>
                <th class="tg-baqh">Number of users</th>
                <th class="tg-baqh">Lag Rate</th>
                <th class="tg-baqh">Quality Rank</th>
              </tr>
     """+ raw_html +"""
    </table>
    </body>
    """ 
    return html

def generate_table(data):
    html = ''
    for index in range(data.shape[0]):
        html += '<tr>'
        for col in range(data.shape[1]):
            html += '<td class="tg-3zjg">'+ str(data.iloc[index,col]) + '</td>'
        html += '</tr>'
    return html 


# root cause analysis table
def generate_html_reason(raw_html):
    html = """
    <style type="text/css">
    .tg  {border-collapse:collapse;border-color:#93a1a1;border-spacing:0;}
    .tg td{background-color:#fdf6e3;border-color:#93a1a1;border-style:solid;border-width:1px;color:#002b36;
      font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
    .tg th{background-color:#657b83;border-color:#93a1a1;border-style:solid;border-width:1px;color:#fdf6e3;
      font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
    .tg .tg-pb0m{border-color:inherit;text-align:center;vertical-align:bottom}
    .tg .tg-9wq8{border-color:inherit;text-align:center;vertical-align:middle}
    .tg .tg-td2w{border-color:inherit;font-size:20px;font-weight:bold;text-align:center;vertical-align:middle}
    .tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
    .tg .tg-vd9z{background-color:#fe0000;border-color:inherit;color:#f8ff00;font-weight:bold;text-align:center;vertical-align:top}
    </style>
    
    <table class="tg">
    <thead>
      <tr>
        <th class="tg-td2w">Dimension</th>
        <th class="tg-td2w">Factor</th>
        <th class="tg-td2w">Users with Lag</th>
        <th class="tg-td2w">Users</th>
        <th class="tg-td2w">Lag Rate</th>
        <th class="tg-td2w">Lag Rate of all users</th>
        <th class="tg-td2w">Influence Degree</th>
        <th class="tg-td2w">Lag Rate after removing this factor</th>
        <th class="tg-td2w">Cause Rank</th>
      </tr>
    </thead>
    <tbody>
    
     """+ raw_html +"""
    <tbody>
    </table>
    """ 
    return html

def generate_table_reason(data):
    html = ''
    for index in range(data.shape[0]):

        html += '<tr>'
        for col in range(data.shape[1]):
            html += '<td class="tg-9wq8">'+ str(data.iloc[index,col]) + '</td>'
        html += '</tr>'
    return html  

# root cause analysis algorithm
def inf_rootcause_function(data,dimension,day):
    '''
    data: original data
    dimension: from which dimension group by
    day: date
    '''
    data = df[df['day']==day]
    data = data.groupby(dimension,as_index=False)[['lag_people','all_people']].sum()
    data['rate'] = data['lag_people']/data['all_people']
    data['value_all'] = data['lag_people'].sum()
    data['cnt_all'] = data['all_people'].sum()
    data['rate_all'] = data['value_all']/data['cnt_all']
    data['inf'] = data['rate_all'] - (data['value_all']-data['lag_people'])/(data['cnt_all']-data['all_people'])
    data['rate_del'] = data['rate_all'] - data['inf']
    daa = data.sort_values('inf',ascending=False)
    data['dimension'] = dimension
    data.columns = ['factor','lag_people','all_people','lag_rate','value_all','cnt_all','rate_all','inf','rate_del','dimension']
    data = data[['dimension','factor','lag_people','all_people','lag_rate','value_all','cnt_all','rate_all','inf','rate_del']]
    return data

# prepare data for making chart of root cause analysis
def rootdata_deal(data,col,value):
    '''
    goal：remove data that has above root cause 
    data：original data from mysql
    col：dimension
    value：root cause
    
    '''
    data = data[((data['day']==day) & (data[col]==value))==False]
    #data = data[(data['day']!=day) & (data[col]!=value)]
    
    data = data.groupby('day',as_index=False)[['lag_people','all_people']].sum()
    data['%s_lag_rate'%col] = data['lag_people']/data['all_people']
    data = data[['day','%s_lag_rate'%col]]
    return data

# make email msg
def msg_make(df,day,text):
    '''
    目的：Encapsulate all the content into msg
    df: original data from mysql
    text：return text from function analyse_textmake
    '''
    
    
    # create email object
    msg = MIMEMultipart()
    # add subject
    subject_content = "Intelligent Data Monitoring Daily Email of Lag Rate of %s"%day
    msg['Subject'] = Header(subject_content)
    # sender
    msg['From'] = mail_sender
    # recipient
    msg['To'] = ",".join(to_list)
    # cc
    #msg['cc'] = ",".join(ccto_list)
    
    # add content
    # text
    text_html = "<h2>" + text + "</h2>"
    
    # table
    df_cdn = analyse_tablemake(df,day)
    table_html = generate_html(generate_table(df_cdn))
    context = MIMEText(text_html+table_html,_subtype='html')
    msg.attach(context)
    
    # pictures
    for num in range(1,6):
        picture_html = MIMEText('<br/><html><body><img src="cid:image%s"></body></html><br/>'%num,'html')
        msg.attach(picture_html)
        msgImage = MIMEImage(open('%s.png'%num, 'rb').read())
        msgImage.add_header('Content-ID','image%s'%num)
        msg.attach(msgImage)
    
    # attached file
    df_today = df[df['day']==day]
    df_today.to_csv('att.csv',index=False,encoding='gbk')
    att = MIMEText(open('att.csv','rb').read(),'base64','gb2312')
    att['Content-Type'] = 'application/octet-stream'
    att["Content-Disposition"] = 'attachment;filename="%sdata_detail.csv"'%day
    msg.attach(att)
    
    if prophet_error(dataprocess(df),day)==1:
        # root cause analysis report - title
        rc_title ='''<h1 style="background-color:#00009b;border-color:inherit;color:#ffffff;font-weight:bold;text-align:left;vertical-align:top">Intelligent Root Cause Analysis Report</h1>'''
        context_rc = MIMEText(rc_title,_subtype='html')
        msg.attach(context_rc)
        
        # table + chart
        
        # lag rate of the day 
        df_kpi = dataprocess(df)
        today_lagrate = round(df_kpi[df_kpi['day']==day]['lag_rate'].values[0],4)
        
        # apply root cause analysis algorithm
        reason_list = list(df.columns[1:6])
        
        data_list =[]
        for columns in reason_list:
            data_list.append(inf_rootcause_function(df,columns,day))
        df_c = pd.concat(data_list)
        df_reason = df_c.sort_values('inf',ascending=False)[:3]
        df_reason = df_reason[['dimension','factor','lag_people','all_people','lag_rate','rate_all','inf','rate_del']]
        df_reason['cause rank'] = ['Top1','Top2','Top3']
        for i in ['lag_rate','rate_all','inf','rate_del']:
            df_reason[i] = round(df_reason[i]*100,2).astype(str) + '%'
        df_reason.columns = ['Dimension','Factor','Users with lag','Users','Lag Rate','Lag Rate of all users','Influence Degree','Lag Rate after removing this factor','Cause Rank']
        
        rc_text = '''
        <style type="text/css">
        tg-bx2p{background-color:#036400;color:#ffffff;font-size:24px;font-weight:bold;}
        tg-s6o7{border-color:inherit;color:#fe0000;font-size:24px;font-weight:bold;text-align:center;vertical-align:bottom}
        </style>
        
        
        <h2>Using<tg-s6o7> timing anomaly detection algorithm Prophet</tg-s6o7>, We found in %s the Lag Rate <tg-s6o7>%s</tg-s6o7> is abnormal. By using <tg-s6o7>root cause analysis algorithm</tg-s6o7>, we found out some possible reasons and hope that relevant colleagues could further check it.</h2>
        '''%(day,str(today_lagrate*100)+'%')
        
        reason_html = generate_html_reason(generate_table_reason(df_reason))
        context_rctext = MIMEText(rc_text+reason_html,_subtype='html')
        msg.attach(context_rctext)
        
        # charts
        
        df_top1 = rootdata_deal(df,df_reason['Dimension'].iloc[0],df_reason['Factor'].iloc[0])
        df_top2 = rootdata_deal(df,df_reason['Dimension'].iloc[1],df_reason['Factor'].iloc[1])
        df_top3 = rootdata_deal(df,df_reason['Dimension'].iloc[2],df_reason['Factor'].iloc[2])
        x_data = list(df_kpi['day'])
        y_data = list(df_kpi['lag_rate'])
        y_data1 = df_top1['%s_lag_rate'%df_reason['Dimension'].iloc[0]]
        y_data2 = df_top2['%s_lag_rate'%df_reason['Dimension'].iloc[1]]
        y_data3 = df_top3['%s_lag_rate'%df_reason['Dimension'].iloc[2]]
        
        line2 = Line(init_opts=opts.InitOpts(theme='light',bg_color=JsCode(bg_color_js),width='700px',height='350px'))
        line2.add_xaxis(x_data)
        line2.add_yaxis('Lag Rate',y_data)
        line2.add_yaxis('Lag Rate after removing Top1 Cause"%s=%s"'%(df_reason['Dimension'].iloc[0],df_reason['Factor'].iloc[0]),y_data1)
        line2.add_yaxis('Lag Rate after removing Top2 Cause"%s=%s"'%(df_reason['Dimension'].iloc[1],df_reason['Factor'].iloc[1]),y_data2)
        line2.add_yaxis('Lag Rate after removing Top3 Cause"%s=%s"'%(df_reason['Dimension'].iloc[2],df_reason['Factor'].iloc[2]),y_data3)
        line2.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        line2.set_global_opts(legend_opts=opts.LegendOpts(pos_bottom='70%',pos_left='10%'),title_opts=opts.TitleOpts(title="Trend Chart of Lag Rate in Recent 20 days"))
        make_snapshot(snapshot,line2.render(),"6.png")
        
        num = 6
        picture_html = MIMEText('<br/><html><body><img src="cid:image%s"></body></html><br/>'%num,'html')
        msg.attach(picture_html)
        msgImage = MIMEImage(open('%s.png'%num, 'rb').read())
        msgImage.add_header('Content-ID','image%s'%num)
        msg.attach(msgImage)
    else:
        pass
    return msg

# send email
def mail_send(msg):
    '''
    goal：send email
    msg：return from funstion msg_make()
    '''
    
    mail_host = "smtp.163.com"
    mail_pass = "*****" 
    
    s = smtplib.SMTP()
    s.connect(mail_host)
    s.login(mail_sender,mail_pass)
    s.sendmail(mail_sender,to_list,msg.as_string())
    print('Success!')
    
# global variables
if __name__ == "__main__":

    bg_color_js = """
    new echarts.graphic.RadialGradient(0.3, 0.3, 0.8, [{
            offset: 0,
            color: '#f7f8fa'
        }, {
            offset: 1,
            color: '#cdd0d5'
        }])"""

    to_list =["290719130@qq.com"]
    #ccto_list = ["290719130@qq.com"]
    mail_sender = "15261816388@163.com"
    day = '2021-02-23'

    # execute
    df = mysql_datagain(day)
    # make text
    text = analyse_textmake(df,day)
    # make pictures and save them
    analyse_picturemake(df,day)
    # make email msg
    msg = msg_make(df,day,text)
    # send email
    mail_send(msg)

<ipython-input-1-7dfdcc550764>:45: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['lag_people'] = df['lag_people'].astype(np.float).astype(int)
<ipython-input-1-7dfdcc550764>:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['all_people'] = df['all_people'].astype(np.float).astype(int)
<ipython-input-1-7dfdcc550764>:45: DeprecationWa

Success!
